https://assets.press.princeton.edu/chapters/s2_10291.pdf

(This is the One-Compartment Model of Repeated Doses, question 3 near end of above pdf)

In [ ]:
import reno

In [ ]:
t = reno.TimeRef()
compartment = reno.Model("one_compartment_model")
compartment.drug_in_system = reno.Stock(doc="mass of medication in the compartment (\"person's blood serum\"?)")
compartment.ingested = reno.Flow(doc="Pulsed inflow of medication when dosage is taken.")
compartment.eliminated = reno.Flow(doc="Rate of change of drug leaving the system.")

compartment.absorption_fraction = reno.Variable(.12)
compartment.dosage = reno.Variable(100 * 1000, doc="Dosage is 100 * 1000 micrograms")
compartment.start = reno.Variable(0, doc="Timestep of first dosage. (in hours)")
compartment.interval = reno.Variable(8, doc="Timesteps between each dosage. (in hours)")

compartment.volume = reno.Variable(3000, doc="Volume of blood serum, 3000 mL") 
compartment.concentration = reno.Variable(compartment.drug_in_system / compartment.volume)
compartment.half_life = reno.Variable(22, doc="Half-life of medication. (in hours)")
compartment.elimination_constant = reno.Variable(-reno.log(0.5) / compartment.half_life)

compartment.eliminated.eq = compartment.elimination_constant * compartment.drug_in_system
compartment.ingested.eq = compartment.absorption_fraction * reno.Piecewise(
    [compartment.dosage, 0], 
    [
        ((t - compartment.start) >= 0) & reno.eq((t - compartment.start) % compartment.interval, 0),
        ((t - compartment.start) < 0) | reno.ne((t - compartment.start) % compartment.interval, 0)
    ],
)

compartment.drug_in_system += compartment.ingested
compartment.drug_in_system -= compartment.eliminated

In [ ]:
compartment.latex()

In [ ]:
ds = compartment(steps=168)

In [ ]:
reno.plot_trace_refs(compartment, [ds], [compartment.concentration])

In [ ]:
compartment.graph()

In [ ]:
compartment.graph(sparklines=True, sparkall=True)